In [158]:
from dataclasses import dataclass
from typing import Tuple
from functools import reduce
from itertools import chain

In [243]:
def clip(b_from, b_to):
    xmin1, xmax1 = b_from
    xmin2, xmax2 = b_to
    return (max(xmin1, xmin2), min(xmax1, xmax2))

def span(b):
    width = b[1] - b[0]
    return width + 1 if width >= 0 else 0

def contains(b, x):
    return b[0] <= x <= b[1]

def overlaps(b1, b2):
    return contains(b2, b1[0]) or contains(b1, b2[1])

@dataclass(frozen=True)
class Cuboid:
    x: Tuple[int, int]
    y: Tuple[int, int]
    z: Tuple[int, int]
        
    def volume(self):
        return span(self.x) * span(self.y) * span(self.z)
    
    def volume_within(self, other):
        return self.intersection(other).volume()
    
    def intersection(self, other):
        return Cuboid(clip(self.x, other.x), clip(self.y, other.y), clip(self.z, other.z))
    
    def overlaps(self, other):
        return self.intersection(other).volume() > 0
    
    def subtract(self, other):
        if not self.overlaps(other):
            return [self]
        xmid = clip(other.x, self.x)
        ymid = clip(other.y, self.y)
        pieces = [
            Cuboid((self.x[0], other.x[0]-1), self.y, self.z),
            Cuboid((other.x[1]+1, self.x[1]), self.y, self.z),
            Cuboid(xmid, (self.y[0], other.y[0]-1), self.z),
            Cuboid(xmid, (other.y[1]+1, self.y[1]), self.z),
            Cuboid(xmid, ymid, (self.z[0], other.z[0]-1)),
            Cuboid(xmid, ymid, (other.z[1]+1, self.z[1])),
        ]
        return [p for p in pieces if p.volume() > 0]

    @staticmethod
    def from_instructions(l):
        limits = [tuple(int(v) for v in rhs.split('..')) for kv in l.split(',') 
                  for rhs in kv.split('=') if '..' in rhs]
        onoff = l.split(' ', 1)[0]
        return Cuboid(*limits), onoff

class CuboidSet:
    def __init__(self, cs=None):
        self.cs = cs or []
        
    def subtract(self, d):
        self.cs = [c for c in chain(*(c.subtract(d) for c in self.cs))]
    
    def add(self, d):
        adds = CuboidSet([d])
        for c in self.cs:
            adds.subtract(c)
        self.cs.extend(adds.cs) 
    
    @staticmethod
    def from_instructions(s):
        cs = [Cuboid.from_instructions(l) for l in s.splitlines()]
        s = CuboidSet()
        for c, onoff in cs:
            if onoff == 'off':
                s.subtract(c)
            else:
                s.add(c)
        return s

def parse_cuboids(s):
    cuboids = []
    onoffs = []
    for l in s.splitlines():
        c, onoff = Cuboid.from_instructions(l)
        cuboids.append(c)
        onoffs.append(onoff)
    return cuboids, onoffs

In [244]:
cs, onoffs = parse_cuboids("""on x=-20..26,y=-36..17,z=-47..7
on x=-20..33,y=-21..23,z=-26..28
on x=-22..28,y=-29..23,z=-38..16
on x=-46..7,y=-6..46,z=-50..-1
on x=-49..1,y=-3..46,z=-24..28
on x=2..47,y=-22..22,z=-23..27
on x=-27..23,y=-28..26,z=-21..29
on x=-39..5,y=-6..47,z=-3..44
on x=-30..21,y=-8..43,z=-13..34
on x=-22..26,y=-27..20,z=-29..19
off x=-48..-32,y=26..41,z=-47..-37
on x=-12..35,y=6..50,z=-50..-2
off x=-48..-32,y=-32..-16,z=-15..-5
on x=-18..26,y=-33..15,z=-7..46
off x=-40..-22,y=-38..-28,z=23..41
on x=-16..35,y=-41..10,z=-47..6
off x=-32..-23,y=11..30,z=-14..3
on x=-49..-5,y=-3..45,z=-29..18
off x=18..30,y=-20..-8,z=-3..13
on x=-41..9,y=-7..43,z=-33..15
on x=-54112..-39298,y=-85059..-49293,z=-27449..7877
on x=967..23432,y=45373..81175,z=27513..53682""")

In [245]:
INIT = (-50, 50)
INIT_C = Cuboid(INIT, INIT, INIT)
count_filled(cs, onoffs, INIT_C)

590784

In [247]:
%%time
with open('../data/day22.txt') as infile:
    s = CuboidSet.from_instructions(infile.read())
    print('[p1] Filled within initial bounds:', sum(c.volume_within(INIT_C) for c in s.cs))
    print('[p2] Filled overall:', sum(c.volume() for c in s.cs))

[p1] Filled within initial bounds 644257
[p2] Filled overall 1235484513229032
CPU times: user 20.6 s, sys: 0 ns, total: 20.6 s
Wall time: 20.6 s
